# EFC 2
## Nome: Miguel Nakajima Marques
## R.A.: 210433

### Questão 3
O objetivo do presente trabalho é propor uma arquitetura de rede que seja melhor que a proposta original apresentada no enunciado do EFC.
Média da taxa de acerto de 5 MLPs na configuração original, como proposto originalmente no enunciado:
```
model = tf.keras.models.Sequential([
 tf.keras.layers.Flatten(),
 tf.keras.layers.Dense(512, activation=tf.nn.relu),
 tf.keras.layers.Dropout(0.5),
 tf.keras.layers.Dense(10, activation=tf.nn.softmax)
])
model.compile(optimizer='adam',
 loss='sparse_categorical_crossentropy',
 metrics=['accuracy'])
model.fit(x_train, y_train, epochs=5)
model.evaluate(x_test, y_test)
```
Taxa de acerto média de 5 MLPs com as configurações originais: 0.9911600112915039

Primeiro vamos importar as bibliotecas que serão utilizadas nessa análise:

In [ ]:
import tensorflow as tf
import os
mnist = tf.keras.datasets.mnist

Agora vamos determinar alguns parâmetros e constantes que serão usados no programa:

In [ ]:
EPOCAS = 20
EPOCAS_ORIGINAL = 5
QUANTIDADE_REDES = 5
DROPOUT = 0.2
QUANTIDADE_NEURONIOS = 512

Vamos carregar as amostras de treinamento e validação em variáveis que serão usadas no programa:

In [ ]:
(x_train, y_train),(x_test,y_test)=mnist.load_data()
x_train,x_test = x_train/255.0, x_test/255.0

Agora vamos configurar a MLP candidata:
* número de camadas intermediárias: 2 
* número de neurônios por camada intermediária: 512
* algoritmo de ajustes de pesos: sparse categorical crossentropy
* taxa de dropout: 0.2
* número de épocas de treinamento: 20

Justificativas para as alterações sugeridas:
* Visto que nas últimas épocas de treinamento do modelo original ainda houve alteração na performance, aumentar o número de épocas de treinamento pode melhorar ainda mais a performance
* Um dropout que causa metade dos neurônios estarem desativados durante uma dada iteração do treinamento diminui pela metade o número de épocas de treinamento que um neurônio será ajustado. Dado um número baixo de épocas de treinamento (5 épocas, no modelo original) isso pode fazer com que um dado neurônio seja ajustado em média somente duas vezes em todo treinamento
Vamos treinar várias redes com a mesma configuração e extrair a média das performances:

In [ ]:
resultados = []
for i in range(QUANTIDADE_REDES):
    
    model = tf.keras.models.Sequential([
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(QUANTIDADE_NEURONIOS,activation=tf.nn.relu),
        tf.keras.layers.Dropout(DROPOUT),
        tf.keras.layers.Dense(QUANTIDADE_NEURONIOS,activation=tf.nn.relu),
        tf.keras.layers.Dropout(DROPOUT),
        tf.keras.layers.Dense(10, activation=tf.nn.softmax)
    ])
    model.compile(optimizer='adam',
                 loss='sparse_categorical_crossentropy',
                 metrics=['accuracy'])

    model.fit(x_train, y_train, epochs=EPOCAS)
    resultados.append(model.evaluate(x_test,y_test))


In [ ]:
print(resultados)

Vamos agora calcular a média dos resultados obtidos:

In [ ]:
media = 0
for resultado in resultados:
    media += resultado[1]/len(resultados)
print(media)

In [ ]:
model_json = model.to_json()
json_file = open("model_MLP.json", "w")
json_file.write(model_json)
json_file.close()
model.save_weights("model_MLP.h5")
print("Model saved to disk")
os.getcwd()

Taxa de acerto média com a nova configuração sugerida: 0.9825999975204467


### Questão 4

Nesa questão iremos abordar o mesmo problema da questão 3 porém utilizando uma rede neural convolucional (CNN).

A rede proposta no enunciado tem performace média entre 5 execuções de 0.9911600112915039

Dado que as bibliotecas e amostras necessárias (tensorflow e keras/mnist) já foram carregas, vamos para as etapas seguintes:

In [ ]:
x_train = x_train.reshape(x_train.shape[0], 28, 28, 1)
x_test = x_test.reshape(x_test.shape[0], 28, 28, 1)
x_train, x_test = x_train / 255.0, x_test / 255.0

Vamos definir o modelo da rede neural a ser utilizada:

In [ ]:
EPOCAS_Q4 = 12
QUANTIDADE_REDES_Q4 = 1

Apesar da performance da rede sugerida no enunciado já ser alta (99,12%), talvez seja possível melhorar ainda mais a performance fazendo alguns ajustes no modelo.

Motivos para as alterações no modelo do enunciado:
* O aumento no número de épocas foi sugerido pois a taxa de acerto entre as épocas ainda estava crescendo entre as épocas 4 e 5, assim sendo, dando mais épocas para o treinamento, é possível que se obtenha uma rede melhor

In [ ]:
resultados_q4 = []
for i in range(QUANTIDADE_REDES_Q4):
    model = tf.keras.models.Sequential()
    model.add(tf.keras.layers.Conv2D(32, kernel_size=(3, 3),
                                     activation='relu',
                                     input_shape=(28, 28, 1)))
    model.add(tf.keras.layers.Conv2D(64, (3, 3), activation='relu'))
    model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))
    model.add(tf.keras.layers.Dropout(0.25))
    model.add(tf.keras.layers.Flatten())
    model.add(tf.keras.layers.Dense(256, activation='relu'))
    model.add(tf.keras.layers.Dropout(0.5))
    model.add(tf.keras.layers.Dense(10, activation='softmax'))

    model.compile(optimizer='adam',
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])

    model.fit(x_train, y_train, epochs=EPOCAS_Q4)
    
    resultados_q4.append(model.evaluate(x_test, y_test))

Vamos calcular a média dos resultados:

In [ ]:
media_q4 = 0
for resultado in resultados_q4:
    media_q4 += resultado[1]
media_q4 /= len(resultados_q4)
print(media_q4)

Vamos salvar os dados do treinamento:

In [ ]:
model_json = model.to_json()
json_file = open("model_CNN.json", "w")
json_file.write(model_json)
json_file.close()
model.save_weights("model_CNN.h5")
print("Model saved to disk")
os.getcwd()

Tabela comparativa das arquitetura:

|Método |performance |
|--------------------|-----------|
|Classificador Linear|   85.05   |
|      MLP           |   98.26   |
|      CNN           |   99.26   |

Pode-se notar que a MLP e CNN obtêm performances parecidas e acima do classificador linear. Assim pode-se concluir que a não linearidade da MLP e da CNN permitem à máquina de classificação se adequar melhor aos dados fornecidos sem que isso cause overfitting.

Considerando o tempo de treinamento, o classificador linear e a CNN levam mais tempo do que a MLP, porém imaginando que a máquina de classificação será treinada somente uma vez antes do seu uso, esse tempo não terá impacto no funcionamento do classificador.